In [1]:
import glob
import nltk
import re

import pandas as pd

from string import punctuation

# Tratamento de dados coletados sobre proposições legislativas

Os dados referentes às proposições legislativas foram coletados manualmente a partir de arquivos estáticos disponíveis em [https://dadosabertos.camara.leg.br/swagger/api.html#staticfile](https://dadosabertos.camara.leg.br/swagger/api.html#staticfile). Os arquivos utilizados nesse projeto estão disponíveis em [../dados/proposicoes](../dados/proposicoes).
Percebemos a necessidade de realizar limpeza dos dados coletados antes de utilizá-los nas análises, então nesse notebook descrevemos o pré-processamento necessário.

Reunimos todos os arquivos em um único dataframe pandas

In [2]:
lista_proposicoes = glob.glob('../dados/proposicoes/*')

In [3]:
tipos_dados = {
    'id': object,
    'uri': object,
    'siglaTipo': object,
    'numero': object,
    'ano': int,
    'codTipo': object,
    'descricaoTipo': object,
    'ementa': object,
    'ementaDetalhada': object,
    'keywords': object,   
    'uriOrgaoNumerador': object,
    'uriPropAnterior': object,
    'uriPropPrincipal': object,
    'uriPropPosterior': object,
    'urlInteiroTeor': object, 
    'urnFinal': object, 
    'ultimoStatus_sequencia': object,
    'ultimoStatus_uriRelator': object, 
    'ultimoStatus_idOrgao': object,
    'ultimoStatus_siglaOrgao': object, 
    'ultimoStatus_uriOrgao': object,
    'ultimoStatus_regime': object, 
    'ultimoStatus_descricaoTramitacao': object,
    'ultimoStatus_idTipoTramitacao': object, 
    'ultimoStatus_descricaoSituacao': object,
    'ultimoStatus_idSituacao': object, 
    'ultimoStatus_despacho': object, 
    'ultimoStatus_url': object
}

tipo_data = ['dataApresentacao', 'ultimoStatus_dataHora']

In [4]:
lista_df = []

for proposicao in lista_proposicoes:
    df_proposicao = pd.read_csv(proposicao, sep=';', dtype=tipos_dados, parse_dates=tipo_data)
    lista_df.append(df_proposicao)

In [5]:
df_proposicao_1934_2021 = pd.concat(lista_df, axis=0, ignore_index=True)
df_proposicao_1934_2021.shape

(680358, 30)

## Seleção de dados referentes aos tipos de proposta legislativa desejados para análise

Selecionaremos apenas as propostas referentes aos seguintes tipos:
- Projeto de Decreto Legislativo [SF] (PDL)
- Projeto de Decreto Legislativo [CD] (PDC)
- Projeto de Decreto Legislativo [CN] (PDN)
- Projeto de Decreto Legislativo [SF] (PDS)
- Proposta de Emenda à Constituição (PEC)
- Projeto de Lei (PL)
- Projeto de Lei da Câmara (PLC)
- Projeto de Lei Complementar (PLP)
- Projeto de Lei de Conversão (PLV)
- Projeto de Resolução da Câmara dos Deputados (PRC)

In [6]:
tipos_proposicoes = ['PDS', 'PDC', 'PDN', 'PEC', 'PL', 'PLC', 'PLP', 'PLV', 'PRC']

In [7]:
df_proposicoes_tipos_desejados = df_proposicao_1934_2021[df_proposicao_1934_2021['siglaTipo'].isin(tipos_proposicoes)].copy()

In [8]:
df_proposicoes_tipos_desejados.shape

(152424, 30)

# Seleção de atributos desejados para análise

In [9]:
df_proposicoes = df_proposicoes_tipos_desejados[['id','siglaTipo','ano', 'codTipo', 'descricaoTipo',
       'ementa', 'ementaDetalhada', 'keywords']].copy()

In [10]:
df_proposicoes.shape

(152424, 8)

# Ajuste de valores faltantes

In [11]:
df_proposicoes.isnull().sum(axis = 0)

id                      0
siglaTipo               0
ano                     0
codTipo                 0
descricaoTipo           0
ementa                  5
ementaDetalhada    109844
keywords              924
dtype: int64

In [12]:
df_proposicoes[
    (df_proposicoes['ementa'].isnull()) & 
    (df_proposicoes['ementaDetalhada'].isnull()) & 
    (df_proposicoes['keywords'].isnull())].head()

,id,siglaTipo,ano,codTipo,descricaoTipo,ementa,ementaDetalhada,keywords
59695,537493,PL,1946,139,Projeto de Lei,NaN,NaN,NaN
71533,347964,PL,1935,139,Projeto de Lei,NaN,NaN,NaN
424157,2123532,PLC,2017,465,Projeto de Lei da Câmara dos Deputados (SF),NaN,NaN,NaN
495917,2308959,PLV,2021,390,Projeto de Lei de Conversão,NaN,NaN,NaN
571961,605771,PL,1954,139,Projeto de Lei,NaN,NaN,NaN


In [13]:
df_proposicoes[(df_proposicoes['ementa'].isnull())].head()

,id,siglaTipo,ano,codTipo,descricaoTipo,ementa,ementaDetalhada,keywords
59695,537493,PL,1946,139,Projeto de Lei,NaN,NaN,NaN
71533,347964,PL,1935,139,Projeto de Lei,NaN,NaN,NaN
424157,2123532,PLC,2017,465,Projeto de Lei da Câmara dos Deputados (SF),NaN,NaN,NaN
495917,2308959,PLV,2021,390,Projeto de Lei de Conversão,NaN,NaN,NaN
571961,605771,PL,1954,139,Projeto de Lei,NaN,NaN,NaN


In [14]:
df_proposicoes.dropna(axis=0, subset=['ementa'], inplace=True)

In [15]:
df_proposicoes.shape

(152419, 8)

# Limpa dados da coluna "keywords"

Identifica propostas legislativas com "keywords"

In [16]:
df_proposicoes_com_keywords = df_proposicoes[df_proposicoes['keywords'].notna()].copy()

In [17]:
df_proposicoes[df_proposicoes['keywords'].notna()]

,id,siglaTipo,ano,codTipo,descricaoTipo,ementa,ementaDetalhada,keywords
105,168293,PDC,1965,135,Projeto de Decreto Legislativo,MANTEM DECISÃO DENEGATORIA DO TRIBUNAL DE CONT...,NaN,"MANUTENÇÃO, DECISÃO, TRIBUNAL DE CONTAS DA UNI..."
106,168297,PDC,1965,135,Projeto de Decreto Legislativo,MANTEM ATO DO TRIBUNAL DE CONTAS DA UNIÃO DE R...,NaN,"MANUTENÇÃO, ATO, TRIBUNAL DE CONTAS DA UNIÃO (..."
107,168300,PDC,1965,135,Projeto de Decreto Legislativo,MANTÉM DECISÃO DENEGATÓRIA DO TRIBUNAL DE CONT...,NaN,"MANUTENÇÃO, DECISÃO, TRIBUNAL DE CONTAS DA UNI..."
108,168303,PDC,1965,135,Projeto de Decreto Legislativo,MANTÉM ATO DO TRIBUNAL DE CONTAS DA UNIÃO QUE ...,NaN,"MANUTENÇÃO, ATO, TRIBUNAL DE CONTAS DA UNIÃO (..."
109,168307,PDC,1965,135,Projeto de Decreto Legislativo,MANTEM O ATO DO TRIBUNAL DE CONTAS DA UNIÃO QU...,NaN,"MANUTENÇÃO, ATO, TRIBUNAL DE CONTAS DA UNIÃO (..."
...,...,...,...,...,...,...,...,...
680339,235344,PRC,1963,141,Projeto de Resolução,AUTORIZA A MESA DA CAMARA DOS DEPUTADOS A EDIT...,NaN,"AUTORIZAÇÃO, MESA DIRETORA, CAMARA DOS DEPUTAD..."
680340,235359,PRC,1963,141,Projeto de Resolução,CRIA COMISSÃO MISTA PARA APRECIAR AS MENSAGENS...,NaN,"CRIAÇÃO, COMISSÃO MISTA, APRECIAÇÃO, MENSAGEM ..."
680341,235375,PRC,1963,141,Projeto de Resolução,PROPÕE MEDIDAS PRELIMINARES PARA A CRIAÇÃO DO ...,NaN,"NORMAS, PROPOSTA, PRELIMINAR, CRIAÇÃO, PARLAME..."
680342,454146,PDC,1963,135,Projeto de Decreto Legislativo,APROVA A CONVENÇÃO SOBRE ASSISTÊNCIA JUDICIÁRI...,NaN,"APROVAÇÃO, ATO INTERNACIONAL, CONVENÇÃO INTERN..."


Download dos pacotes realtivos a "stopwords" e pontuação da biblioteca NLTK

In [18]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/cecivieira/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/cecivieira/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Remove pontuação, preposições e artigos (stopwords)

In [19]:
meses = ['janeiro', 'fevereiro', 'março', 'abril', 'maio', 'junho', 'julho','agosto', 'setembro', 'outubro', 'novembro', 'dezembro']

In [20]:
def define_stopwords_punctuation():
    stopwords = nltk.corpus.stopwords.words('portuguese') + meses
    pontuacao = list(punctuation)
    stopwords.extend(pontuacao)
    return stopwords

Adiciona a `keywords` toda palavra que não for uma stopword ou número

In [22]:
def remove_stopwords_punctuation_da_sentenca(texto):
    padrao_digitos = r'[0-9]'
    texto = re.sub(padrao_digitos, '', texto)
    palavras = nltk.tokenize.word_tokenize(texto.lower())
    stopwords = define_stopwords_punctuation()
    keywords = [palavra for palavra in palavras if palavra not in stopwords]
    return keywords

In [23]:
df_proposicoes_com_keywords['keywords'] = df_proposicoes_com_keywords['keywords'].apply(remove_stopwords_punctuation_da_sentenca)

Converte lista para string

In [24]:
def converte_lista_string(lista):
    return ','.join([palavra for palavra in lista])

In [25]:
df_proposicoes_com_keywords['keywords'] = df_proposicoes_com_keywords['keywords'].apply(converte_lista_string)

Retira do dataframe proposições cujas `keywords` ficaram vazias depois da limpeza

In [26]:
df_proposicoes_com_keywords = df_proposicoes_com_keywords[df_proposicoes_com_keywords['keywords'] != '']

In [27]:
df_proposicoes_com_keywords.head()

,id,siglaTipo,ano,codTipo,descricaoTipo,ementa,ementaDetalhada,keywords
105,168293,PDC,1965,135,Projeto de Decreto Legislativo,MANTEM DECISÃO DENEGATORIA DO TRIBUNAL DE CONT...,NaN,"manutenção,decisão,tribunal,contas,união,tcu,d..."
106,168297,PDC,1965,135,Projeto de Decreto Legislativo,MANTEM ATO DO TRIBUNAL DE CONTAS DA UNIÃO DE R...,NaN,"manutenção,ato,tribunal,contas,união,tcu,recus..."
107,168300,PDC,1965,135,Projeto de Decreto Legislativo,MANTÉM DECISÃO DENEGATÓRIA DO TRIBUNAL DE CONT...,NaN,"manutenção,decisão,tribunal,contas,união,tcu,d..."
108,168303,PDC,1965,135,Projeto de Decreto Legislativo,MANTÉM ATO DO TRIBUNAL DE CONTAS DA UNIÃO QUE ...,NaN,"manutenção,ato,tribunal,contas,união,tcu,negaç..."
109,168307,PDC,1965,135,Projeto de Decreto Legislativo,MANTEM O ATO DO TRIBUNAL DE CONTAS DA UNIÃO QU...,NaN,"manutenção,ato,tribunal,contas,união,tcu,deneg..."


# Extração de palavras chaves das ementas, quando necessário

Verificamos que algumas propostas legislativas não possuem palavras chave desde sua coleta, por isso extrairemos essas palavras do campo `ementa`.

Identificação de propostas legislativas com campo "keywords" vazio

In [28]:
df_proposicoes_sem_keywords = df_proposicoes[df_proposicoes['keywords'].isna()].copy()

Remoção de pontuação, preposições e artigos (stopwords)

In [29]:
df_proposicoes_sem_keywords['keywords'] = df_proposicoes_sem_keywords['ementa'].apply(remove_stopwords_punctuation_da_sentenca)

Identifica caracteres e abreviações semanticamente irrelevantes ainda presentes na coluna "keywords"

In [30]:
lista_keywords = []
lista_keywords_temp = df_proposicoes_sem_keywords['keywords'].tolist()
_ = [lista_keywords.extend(item) for item in lista_keywords_temp]

In [31]:
palavras_para_descarte = [item for item in set(lista_keywords) if len(item) <= 3]

Retira os substantivos da lista de caracteres e abreviações semanticamente irrelevantes

In [32]:
substantivos_nao_descartaveis = ['cão', 'mãe', 'oab', 'boa', 'pré', 'voz', 'rui', 'uva', 'gás', 'glp', 'apa']

Remove da coluna "keywords" lista de caracteres e abreviações semanticamente irrelevantes

In [33]:
palavras_para_descarte_refinada = [palavra for palavra in palavras_para_descarte if palavra not in substantivos_nao_descartaveis]

In [34]:
def remove_palavras_para_descarte_da_sentenca(texto):
    keywords = []
    for palavra in texto:
        if palavra not in palavras_para_descarte_refinada:
            keywords.append(palavra)
    return keywords

In [35]:
df_proposicoes_sem_keywords['keywords'] = df_proposicoes_sem_keywords['keywords'].apply(remove_palavras_para_descarte_da_sentenca)

Identifica, na coluna "keywords", palavras sem relevancia semântica, por exemplo: "altera", "dispõe" e "sobre".

In [36]:
def gera_n_grams(texto, ngram=2):
    temporario = zip(*[texto[indice:] for indice in range(0,ngram)])
    resultado = [' '.join(ngram) for ngram in temporario]
    return resultado

In [37]:
df_proposicoes_sem_keywords['bigrams'] = df_proposicoes_sem_keywords['keywords'].apply(gera_n_grams)

In [38]:
lista_ngrams = []
lista_ngrams_temp = df_proposicoes_sem_keywords['bigrams'].tolist()
_ = [lista_ngrams.extend(item) for item in lista_ngrams_temp]

In [39]:
bigrams_comuns = nltk.FreqDist(lista_ngrams).most_common(50)

In [40]:
lista_bigramas_comuns = [bigrama for bigrama, frequencia in bigrams_comuns]

Foram analisados os 50 bigramas mais frequentes e identificados os semanticamente irrelevantes para criacao de `keywords`

In [41]:
lista_bigramas_comuns_limpa = ['dispõe sobre', 'outras providências', 'nova redação', 'poder executivo', 'distrito federal',
 'autoriza poder', 'federal outras','redação constituição', 'dispõe sôbre', 'código penal', 'artigo constituição',
 'disposições constitucionais', 'altera dispõe', 'decreto-lei código', 'constitucionais transitórias', 'altera redação',
 'abre ministério', 'executivo abrir', 'redação artigo', 'sobre criação', 'acrescenta parágrafo', 'parágrafo único',
 'concede isenção', 'altera dispositivos', 'altera complementar', 'dispondo sobre', 'código processo', 'outras providências.',
 'providências. historico', 'ministério fazenda', 'altera leis', 'programa nacional', 'quadro permanente', 'outras providencias',
 'inciso constituição', 'abrir ministério', 'estabelece normas', 'ministério justiça', 'tempo serviço', 'instituto nacional',
 'institui sistema', 'operações crédito', 'altera institui', 'dispõe sôbre']

In [42]:
palavras_para_descarte_origem_bigramas = []
_ = [palavras_para_descarte_origem_bigramas.extend(bigrama.split(' ')) for bigrama in lista_bigramas_comuns_limpa]
palavras_para_descarte_origem_bigramas_unicas = set(palavras_para_descarte_origem_bigramas)

Remove palavras irrelevantes originarias dos bigramas

In [43]:
def remove_palavras_origem_bigramas_da_sentenca(texto):
    keywords = []
    for palavra in texto:
        if palavra not in palavras_para_descarte_origem_bigramas_unicas:
            keywords.append(palavra)
    return keywords

In [44]:
df_proposicoes_sem_keywords['keywords'] = df_proposicoes_sem_keywords['keywords'].apply(remove_palavras_origem_bigramas_da_sentenca)

Converte lista para string

In [45]:
df_proposicoes_sem_keywords['keywords'] = df_proposicoes_sem_keywords['keywords'].apply(converte_lista_string)

Elimina coluna "bigrams"

In [46]:
df_proposicoes_sem_keywords = df_proposicoes_sem_keywords.drop(columns=['bigrams'])

Remove propostas cujo campo "keywords" ficou vazio após a limpeza da ementa e extração de palavras chaves

In [47]:
df_proposicoes_sem_keywords = df_proposicoes_sem_keywords[df_proposicoes_sem_keywords['keywords'] != '']

In [48]:
df_proposicoes_sem_keywords[df_proposicoes_sem_keywords['keywords']== ''] 

,id,siglaTipo,ano,codTipo,descricaoTipo,ementa,ementaDetalhada,keywords


# Reuni dados em um único dataframe

In [49]:
df_proposicoes_v_final = pd.concat([df_proposicoes_com_keywords, df_proposicoes_sem_keywords])

In [50]:
df_proposicoes_v_final.shape

(152386, 8)

In [51]:
df_proposicoes_v_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 152386 entries, 105 to 680357
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   id               152386 non-null  object
 1   siglaTipo        152386 non-null  object
 2   ano              152386 non-null  int64 
 3   codTipo          152386 non-null  object
 4   descricaoTipo    152386 non-null  object
 5   ementa           152386 non-null  object
 6   ementaDetalhada  42563 non-null   object
 7   keywords         152386 non-null  object
dtypes: int64(1), object(7)
memory usage: 10.5+ MB


In [52]:
df_proposicoes_v_final.to_csv('../dados/proposicoes_legislativas_limpas.csv', index=False)